In [ ]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import geopandas as gpd
import pandas as pd
import branca
from siuba import *

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
import bb_rt_helper

In [ ]:
import importlib
importlib.reload(bb_rt_helper)

# 100 Recs - D11

## 1 - Corridor Improvements

In [ ]:
bb_rt_helper.fm_from_bbutils?

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='11 - San Diego', category='corridor')

In [ ]:
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
m_corr = fm_dict[278].segment_speed_map()

In [ ]:
m_corr

## Draft Corridor Recs, SDMTS, D11

* Entire Route 28 (nn mi, nn speed-based delay, nn schedule-based delay)
* Route 10, Goldfinch to 54 ()
    * paralleled by faster Rapid 215 E of Park
* Route 27, Mission to Moraga
* Route 12, Broadway to Euclid

## Route 28

In [ ]:
shape_id = '28_2_48'
stop_range = [2, 18]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['28'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_28 = fm_dict[278].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors = [corr_28]

## Route 28

In [ ]:
shape_id = '815_2_21'
stop_range = [2, 18]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].reset_filter()

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_815 = fm_dict[278].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors += [corr_815]

## Route 10

In [ ]:
shape_id = '10_2_68'
stop_range = [6, 21]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['10'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_10 = fm_dict[278].corridor.copy()
corridors += [corr_10]

## Route 27

In [ ]:
shape_id = '27_2_81'
stop_range = [3, 16]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['27'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_27 = fm_dict[278].corridor.copy()
corridors += [corr_27]

## Route 12

In [ ]:
shape_id = '12_2_13'
stop_range = [3, 31]

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].set_filter(route_names=['12'])

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
corr_12 = fm_dict[278].corridor.copy()
corridors += [corr_12]

## 2 - Hotspot Improvements

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='11 - San Diego', category='hotspot', fm_dict = fm_dict)

In [ ]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

In [ ]:
m_hs = fm_dict[278].segment_speed_map()

In [ ]:
m_hs

# Draft Hotspot Recs, SDMTS, D11

In [ ]:
0.0 == 0

## Route 27 (Genesee)

In [ ]:
shape_id = '27_2_81'
stop_range = [19, 23]

In [ ]:
fm_dict[278].set_filter(route_names=['27'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_27 = fm_dict[278].corridor.copy()
hotspots = [hs_27]

## Route 10 (I-5)

In [ ]:
shape_id = '10_3_67'
stop_range = [21, 22]

In [ ]:
fm_dict[278].set_filter(route_names=['10'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_10 = fm_dict[278].corridor.copy()
hotspots += [hs_10]

## Route 35

In [ ]:
shape_id = '35_3_50'
stop_range = [5, 9]

In [ ]:
fm_dict[278].reset_filter()

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_35 = fm_dict[278].corridor.copy()
hotspots += [hs_35]

## Route 14 (I-8)

In [ ]:
shape_id = '14_2_96'
stop_range = [28, 29]

In [ ]:
fm_dict[278].set_filter(route_names=['14'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_14 = fm_dict[278].corridor.copy()
hotspots += [hs_14]

## Route 3 (Commercial to 24th)

In [ ]:
shape_id = '3_0_90'
stop_range = [27, 32]

In [ ]:
fm_dict[278].set_filter(route_names=['3'])

In [ ]:
fm_dict[278].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[278].corridor_metrics()

In [ ]:
fm_dict[278].quick_map_corridor()

In [ ]:
hs_3 = fm_dict[278].corridor.copy()
hotspots += [hs_3]

## Map Testing

In [ ]:
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [ ]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = '11 - San Diego')